# Spherical harmonics vs graph Fourier modes

[Nathanaël Perraudin](http://perraudin.info), [Michaël Defferrard](http://deff.ch), Tomasz Kacprzak

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from time import time

import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import pygsp as pg

from scnn import utils

In [ ]:
plt.rcParams['figure.figsize'] = (17, 5)  # (9, 4) for matplotlib notebook

nside = 16

## 1 Spherical harmonics

* Spherical harmonics are indexed by degree (or angular frequency) $\ell$ and order $m \in [0, \ell]$.
* $l=0$ is the constant, $l=1$ is the two monopoles, $l=2$ is the three dipoles, etc.
* Coefficients are commonly defined as $a_{\ell m}$.
* A spherical harmonic takes value $Y_\ell^m(\theta, \phi)$ in angular direction $(\theta, \phi)$.

In [ ]:
def plot_spherical_harmonic(nside, l, m):

    lmax = l
    idx = hp.sphtfunc.Alm.getidx(lmax, l, m)
    size = hp.sphtfunc.Alm.getsize(lmax, mmax=lmax)
    print(f'{size} spherical harmonics for l in [0, {lmax}]')
    print('l={}, m={} is at index {}'.format(l, m, idx))

    alm = np.zeros(size, dtype=np.complex128)
    alm[idx] = 1

    map = hp.sphtfunc.alm2map(alm, nside, lmax, verbose=False)
    hp.mollview(map, title=f"Spherical harmonic of degree l={l} and order m={m}")
    #hp.cartview(map, title=f"Spherical harmonic l={l}, m={m}")

plot_spherical_harmonic(nside, l=2, m=0)

Compute the spherical harmonics up to $\ell_{max}$ and plot them.

In [ ]:
def compute_spherical_harmonics(nside, lmax):
    """Compute the spherical harmonics up to lmax.
    
    Returns
    -------
    harmonics: array of shape n_pixels x n_harmonics
        Harmonics are in nested order.
    """

    n_harmonics = np.sum(np.arange(1, lmax+2))
    harmonics = np.empty((hp.nside2npix(nside), n_harmonics))
    midx = 0

    for l in range(lmax+1):
        for m in range(l+1):
            size = hp.sphtfunc.Alm.getsize(l, mmax=l)
            alm = np.zeros(size, dtype=np.complex128)
            idx = hp.sphtfunc.Alm.getidx(l, l, m)
            alm[idx] = 1
            harmonic = hp.sphtfunc.alm2map(alm, nside, l, verbose=False)
            harmonics[:, midx] = hp.reorder(harmonic, r2n=True)
            midx += 1

    return harmonics

harmonics = compute_spherical_harmonics(nside, lmax=18)
harmonics.shape

In [ ]:
def plot_spherical_harmonics(harmonics):
    n_harmonics = harmonics.shape[1]
    l, m = 0, 0
    for idx in range(n_harmonics):
        hp.mollview(harmonics[:, idx], 
                    title='Spherical harmonic l={}, m={}'.format(l, m),
                    nest=True,
                    sub=(np.sqrt(n_harmonics), np.sqrt(n_harmonics), idx+1),
                    max=np.max(np.abs(harmonics[:, :n_harmonics])),
                    min=-np.max(np.abs(harmonics[:, :n_harmonics])),
                    cbar=False)
        m += 1
        if m > l:
            l += 1
            m = 0

plot_spherical_harmonics(harmonics[:, :16])

## 2 Graph Fourier modes

The graph Fourier modes are the eigenvectors of the graph Laplacian.
$$L = U \Lambda U^T$$

In [ ]:
graph = utils.healpix_graph(nside, lap_type='normalized', nest=True, dtype=np.float64)
graph.compute_fourier_basis()

The weighted adjacency matrix is very sparse. Distance between pixels is not constant.

In [ ]:
fig, axes = plt.subplots(1, 2)
axes[0].imshow(graph.W.toarray())
axes[1].hist(graph.W.data);

Fourier modes of the graph.

In [ ]:
def plot_fourier_modes(eigenvectors):
    n_eigenvectors = eigenvectors.shape[1]
    l, m = 0, 0
    for idx in range(n_eigenvectors):
        hp.mollview(eigenvectors[:, idx], 
                    title='Eigenvector {}, e={:.2e} (l={}, m={})'.format(idx, graph.e[idx], l, m),
                    nest=True,
                    sub=(np.sqrt(n_eigenvectors), np.sqrt(n_eigenvectors), idx+1),
                    max=np.max(np.abs(eigenvectors[:, :n_eigenvectors])),
                    min=-np.max(np.abs(eigenvectors[:, :n_eigenvectors])),
                    cbar=False)
        m += 1
        if m > l:
            l += 1
            m = -l

plot_fourier_modes(graph.U[:, :16])

The eigenvalues are clearly organized in groups, which corresponds to angular frequencies $\ell$ of the spherical harmonics.

In [ ]:
plt.plot(graph.e[:50], '.-');